In [ ]:
from tqdm import tqdm

In [ ]:
pdf_path = "./pdfs/DR--185549702_INTRO.pdf"

# 1. LLMSherpa (LayoutPDFReader)

In [ ]:
# !pip install llmsherpa

In [ ]:
from llmsherpa.readers import LayoutPDFReader

In [ ]:
llmsherpa_api_url = "https://readers.llmsherpa.com/api/document/developer/parseDocument?renderFormat=all"

pdf_reader = LayoutPDFReader(llmsherpa_api_url)
doc = pdf_reader.read_pdf(pdf_path)

### HTML extraction

In [ ]:
doc.to_html()

### Text extraction

In [ ]:
doc.to_text()

### JSON

In [ ]:
import pandas as pd
pd.DataFrame(doc.json).head()

### Tables

In [ ]:
doc.tables()

In [ ]:
doc.tables()[-1].to_html()

### Chunk-wise extraction

In [ ]:
all_chunks = []
for chunk in doc.chunks():
    all_chunks.append(chunk.to_text())

In [ ]:
all_chunks[-2]

# 2. PyPDF

In [ ]:
# !pip install pypdf

In [ ]:
from pypdf import PdfReader

In [ ]:
# creating a pdf reader object 
reader = PdfReader(pdf_path) 

# printing number of pages in pdf file 
print(f"No. of pages = {len(reader.pages)}\n\n") 

# getting a specific page from the pdf file 
page = reader.pages[0] 

# extracting text from page 
text = page.extract_text() 
print(text)

In [ ]:
all_texts = []

for page in tqdm(reader.pages):
    text = page.extract_text()
    all_texts.append(text)
    
raw_text = "\n".join(all_texts)
print(raw_text)

### Last Page Table

In [ ]:
print(all_texts[-1])

# 3. PDFPlumber

### Not too great at text extraction, but can extract tables!

In [ ]:
# !pip install pdfplumber

In [ ]:
import pdfplumber

In [ ]:
with pdfplumber.open(pdf_path) as pdf:
    first_page = pdf.pages[0]
    print(first_page.extract_text())

In [ ]:
all_texts = []

with pdfplumber.open(pdf_path) as pdf:
    for page in tqdm(pdf.pages):
        text = page.extract_text()
        all_texts.append(text)
    
raw_text = "\n".join(all_texts)
print(raw_text)

In [ ]:
def extract_table_to_html(pdf_path, page_number=0, table_index=0):
    with pdfplumber.open(pdf_path) as pdf:
        page = pdf.pages[page_number]
        tables = page.extract_tables()
        if len(tables) > table_index:
            table = tables[table_index]
            html = "<table>"
            for row in table:
                html += "<tr>"
                for cell in row:
                    html += "<td>" + str(cell) + "</td>"
                html += "</tr>"
            html += "</table>"
            return html
        else:
            return "No table found at index {} on page {}".format(table_index, page_number)


html_table = extract_table_to_html(pdf_path, page_number=-1)
print(html_table)

# 4. Tika

### Ignore

In [ ]:
# !pip install tika

In [ ]:
from tika import parser

In [ ]:
raw = parser.from_file(pdf_path)
print(raw['content'])